<a href="https://colab.research.google.com/github/yoshihiroo/programming-workshop/blob/master/QC4U_2022/qc4uchapter4_cirq_English.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# QC4U Day4 Cirq porting
2022.10.15 Updated.

This is an attempt to porting of the [QC4U](https://altema.is.tohoku.ac.jp/QC4U/) code written by Prof. Ohzeki of Tohoku University into Cirq for my recap and understanding. I am stealing with pride the almost all text of the explanation from the original site. (The article has been published with Prof. Ohzeki's permission.)

[The original code of Day 4](https://colab.research.google.com/gist/mohzeki222/202865a9fcc45bd37dc2e124d9ab0a84/qc4uchapter4.ipynb)

# Installing Cirq

Without further ado, let's explore the possibilities of quantum computing again this time!
First, install Cirq provided by Google.

In [ ]:
pip install cirq

First, prepare the modules you always use.

In [ ]:
import cirq

### Previous Recap.

So far we have learned H, X, Z, and the control Z-gate.
Each of them is characterized by H = superposition, X = inversion, and Z = scratch (negative sign for only |1>).
And the control Z-gate was to scratch only|11>.
By combining these features to create a qubit interaction, we were able to simulate what was happening within matter.

For example, in a magnet, spins, which are like pieces of a magnet, are constantly changing direction.
However, the interactions between neighboring spins, which try to align their orientations, tend to align the overall orientation.
The overall orientation tends to be aligned.
Some of the factors that interfere with this are of environmental origin, such as thermal fluctuations, while others can be prevented through the manipulation of the spins by toppling their orientation.

Let us consider a quantum circuit that simulates such a situation.
You created that interaction last time.
Let us assume that the spin (piece of a magnet) of a qubit is upward directed at the qubit's |0> and downward directed at the qubit's |1>, and let us assume that the spin of the qubit is parallel or antiparallel to the spin of the magnet.
We created a circuit such that the probability amplitude remains the same when each direction is aligned parallel or antiparallel, and is negative when the direction is upside down.

In [ ]:
qc = cirq.Circuit()
q = cirq.LineQubit.range(2)

qc.append(cirq.CNOT(q[0], q[1]))
qc.append(cirq.Z(q[1]))
qc.append(cirq.CNOT(q[0], q[1]))

When you looked at the circuit, it was print or SVGCircuit.

In [ ]:
#print(qc)
from cirq.contrib.svg import SVGCircuit
SVGCircuit(qc)

If you want to see what kind of results you will get, run the simulation.
We will again use our own function that we created last time.

In [ ]:
import numpy as np
def sim_state(qc,disp=True):
  res = sim.simulate(qc)
  if disp == True:
    print(cirq.dirac_notation(np.array(res.final_state_vector)))
  return res

This time, since we are also interested in the result of which state will eventually be output, let's provide a function that can also display a histogram based on that probability.
So, let's prepare a function that can also display a histogram based on that probability.

In [ ]:
def sim_state_exp(qc):
  qc.append(cirq.measure(q, key='m'))
  res = sim.run(qc, repetitions=1000)
  counts = res.histogram(key='m')
  return res

In [ ]:
sim = cirq.Simulator()
state = sim_state(qc)

In [ ]:
import matplotlib.pyplot as plt

def binary_labels(num_qubits):
    return [bin(x)[2:].zfill(num_qubits) for x in range(2 ** num_qubits)]

ans = sim_state_exp(qc)
cirq.plot_state_histogram(ans, plt.subplot(), tick_label=binary_labels(2))
plt.show()

The initial condition of the qubit input to the quantum circuit is |00>, so the result was returned as is.
To find out what happens when various inputs are put in, we can use the Hadamard circuit.

In [ ]:
qc2 = cirq.Circuit()
q = cirq.LineQubit.range(2)


#2 qubits in superposition state
qc2.append(cirq.H.on_each(q))

qc2.append(cirq.CNOT(q[0], q[1]))
qc2.append(cirq.Z(q[1]))
qc2.append(cirq.CNOT(q[0], q[1]))

In [ ]:
SVGCircuit(qc2)

In [ ]:
state2 = sim_state(qc2)

As we had hoped, the sign of the probability amplitude over the|00> and|11> remained the same, and when the two qubits were different, as in the case of|01> and|10>, the sign was reversed.

Inside matter, microscopic objects on the atomic scale evolve in time according to quantum mechanics.
The rules are quite simple: just multiply the exponential function by an imaginary number, energy, and time.

If both spins are aligned, the energy goes down (the spin thinks that's a good deal), and if they are mutually exclusive, the energy goes up (the spin thinks that's a loss). Consider such a situation. This is the Ising model, known as the standard model of magnets.
In this case, let's make the Z-circuit a rotating Z-circuit and vary its angle with time.

In [ ]:
qc3 = cirq.Circuit()
q = cirq.LineQubit.range(2)

theta = 0.3

qc3.append(cirq.H.on_each(q))

qc3.append(cirq.CNOT(q[0], q[1]))
qc3.append(cirq.rz(theta).on(q[1]))
qc3.append(cirq.CNOT(q[0], q[1]))

In [ ]:
SVGCircuit(qc3)

In [ ]:
state3 = sim_state(qc3)

When the qubits (spins) are aligned, the imaginary part is (-), and when they are mutually different, the imaginary part is (+), showing the difference, right?
This indicates that the probability amplitude changes depending on the energy, and the direction of the amplitude depends on the sign of the energy.

### Eigenstates and Quantum Calculations

Next, let's consider manipulating the spin orientation.
This is where the X-gate, or rotational X-gate, comes in.

If we continue to apply this, we will constantly change the orientation of the object to|0> and|1>.


In [ ]:
qc4 = cirq.Circuit()
q = cirq.LineQubit.range(1)

theta = 0.3

qc4.append(cirq.rx(theta).on(q[0]))

In [ ]:
SVGCircuit(qc4)

In [ ]:
state4 = sim_state(qc4)

In [ ]:
from cirq_web import BlochSphere
display(BlochSphere(state_vector=cirq.to_valid_state_vector(state4.final_state_vector)))

We can see that the state is gradually transitioning from|0> to|1>.
The way this probability amplitude changes is also characteristic.

As we saw earlier, the probability amplitude was preserved as it is when the initial condition is |00>.
Also, even if the Hadamard circuit is used to superimpose the probability amplitude on the condition of|00>, |01>, |10>, and |11>, the magnitude of each of them does not actually change.
In other words, the probability of occurrence of the result remains the same.
In this sense, the state of the system does not change.
Such a state is called an eigenstate, and each quantum circuit has its own eigenstate.

On the other hand, the rotating X-gate (and X-gate) changes the probability amplitude of the |0> state and transfers it to the other state, |1>.
This indicates that for a rotating X-gate, neither|0> nor|1> is an eigenstate.

In quantum computation, we need to hurt the probability amplitude, and
and reducing the probability amplitude and transitioning to another state is skillfully used to achieve the desired result.
The key concept is whether it is an eigenstate or not.
The states |0> and |1> are eigenstates with respect to Z-related effects.
Therefore, another action, such as an X-gate, is needed to move the qubit or change the probability amplitude.

Incidentally, the X-gate has the superposition state as an eigenstate.

In [ ]:
qc5 = cirq.Circuit()
q = cirq.LineQubit.range(1)

theta = 0.3

qc5.append(cirq.H(q[0]))
qc5.append(cirq.rx(theta).on(q[0]))

In [ ]:
SVGCircuit(qc5)

In [ ]:
state5 = sim_state(qc5)

It is an eigenstate because we are only changing the probability amplitude while keeping the cohesion of the superposition state of |0>+|1>.

So what happens when we apply a rotating Z-gate to this quantum state?

In [ ]:
qc6 = cirq.Circuit()
q = cirq.LineQubit.range(1)

theta = 0.3

qc6.append(cirq.H(q[0]))
qc6.append(cirq.rz(theta).on(q[0]))

In [ ]:
SVGCircuit(qc6)

In [ ]:
state6 = sim_state(qc6)

The respective magnitudes of the probability amplitudes do not change because each of the |0> and |1> is an eigenstate of the Z-gate.
However, the superposition is broken.
They are not the same coefficients.
When the Hadamard circuit is applied to the superposition state, it returns to |0>.
You could use that to see the degree of superposition state.
We did the same thing with Grover's algorithm.
Now let's apply the Hadamard circuit.

In [ ]:
qc6.append(cirq.H(q[0]))


In [ ]:
state6 = sim_state(qc6)

The collapse of the superposition has affected the state of the overlap, and the state of |1> has popped up.
From the superposition state, a rotating Z-gate was applied to collapse it.
After going through the Hadamard circuit, we were able to separate the part of the superposition that was holding and the part that was collapsing.
The Hadamard circuit corresponds to a 90-degree rotation when expressed in terms of a rotational X-gate.
Therefore, the rotating X-gate is used to preserve the part of the superposition state and separate the other parts.


In [ ]:
qc7 = cirq.Circuit()
q = cirq.LineQubit.range(1)

theta = 0.3

qc7.append(cirq.H(q[0]))
qc7.append(cirq.rz(theta).on(q[0]))
qc7.append(cirq.rx(theta).on(q[0]))

In [ ]:
SVGCircuit(qc7)

In [ ]:
state7 = sim_state(qc7)

Calculating the square of the magnitude of the amplitude, we find that the value for|0> is about 0.535 and that for|1> is about 0.465.
The probability of occurrence is slightly higher for|0>.

In [ ]:
ans = sim_state_exp(qc7)
cirq.plot_state_histogram(ans, plt.subplot(), tick_label=binary_labels(1))
plt.show()

The state of|0> is taken out of the superposition of|0> and|1>.
This suggests that Rz is squeezing out the state of the overlap.
Rx is responsible for pushing out the other collapsed states while leaving the superposition intact.
Rz is collapsing the amplitudes of the overlap states of |0> and |1>.

### Quantum Annealing

Now recall that Rz is a rotation, and that the angle of rotation corresponds to the energy of a qubit or spin in quantum mechanics.
A qubit pointing upward now has its probability amplitude unchanged, while a qubit pointing downward now has a negative probability amplitude. Interpreting this in the same way as before, let's think that the upward direction corresponds to a situation where the energy goes down (a bargain) and the downward direction corresponds to a situation where the energy goes up (a loss).
Then we see the gradual emergence of the upward qubit state, which is the profitable state.

An algorithm that takes advantage of this to extract the lowest energy ground state is called
quantum annealing.
In quantum annealing.
At first, the action of the rotating X-gate is kept strong and the Z-related gate is kept weak (superposition state), and
Finally, the action of the rotating X-gate is weakened and the Z-related gate is strengthened.


In [ ]:
qc8 = cirq.Circuit()
q = cirq.LineQubit.range(1)

theta = 0.1

qc8.append(cirq.H(q[0]))

Tall = 100
for k in range(Tall):
  qc8.append(cirq.rz(theta*k/Tall).on(q[0]))
  qc8.append(cirq.rx(theta*(1-k/Tall)).on(q[0]))

In [ ]:
SVGCircuit(qc8)

In [ ]:
state8 = sim_state(qc8)

In [ ]:
ans8 = sim_state_exp(qc8)
cirq.plot_state_histogram(ans8, plt.subplot(), tick_label=binary_labels(1))
plt.show()

Only the well|0> states can be obtained with large probability.
The same can be done for multiple qubits.


In [ ]:
qc9 = cirq.Circuit()
q = cirq.LineQubit.range(2)

theta = 0.2

qc9.append(cirq.H.on_each(q))

Tall = 100
for k in range(Tall):
  qc9.append(cirq.rx(theta*(1-k/Tall)).on_each(q))

  qc9.append(cirq.CNOT(q[0], q[1]))
  qc9.append(cirq.rz(theta*k/Tall).on(q[1]))
  qc9.append(cirq.CNOT(q[0], q[1]))

In [ ]:
SVGCircuit(qc9)

In [ ]:
state9 = sim_state(qc9)

In [ ]:
ans9 = sim_state_exp(qc9)
cirq.plot_state_histogram(ans9, plt.subplot(), tick_label=binary_labels(2))
plt.show()

As you aim, you will get either |00> or |11> with very high probability.

We should be able to use this for general Ising models.
Even this simple one of|0> and|1>, if we expand its interpretation, we can come up with so many applications.
It is better to be left or right instead of up or down, or to let a quantum computer choose, etc.
Or, "Should I choose a science course or a liberal arts course? Let the quantum computer make the choice, for example.
It is not just one of the two options, but many other factors are involved, and it is difficult to decide what is the appropriate choice. In such cases, just as with energy, there is or can be set a numerical indicator that shows which is preferable, and the best choice is taken out.
Mathematical problems with such a goal are called combinatorial optimization problems.
Quantum annealing can solve such combinatorial optimization problems.


In [ ]:
class QA(cirq.Gate):
  def __init__(self,J,h,s1,s2):
    self.J = J
    self.h = h
    self.s1 = s1
    self.s2 = s2
    self.n = len(self.h)

  def _num_qubits_(self):
    return self.n

  def _decompose_(self, qubits):
    q = qubits

    #Rotating X-gate for entire qubit
    yield cirq.rx(self.s1).on_each(q)

    #Rotating Z-gate etc. based on J and h throughout the qubit
    for i in range(self.n):
      yield cirq.rz(self.s2*self.h[i]).on(q[i])

    for i in range(self.n):
      for j in range(self.n):
        if i != j:
          yield cirq.CNOT(q[i], q[j])
          yield cirq.rz(self.s2*J[i,j]).on(q[j])
          yield cirq.CNOT(q[i], q[j])

  def _circuit_diagram_info_(self, args):
    return ["Uqubo"] * self.num_qubits()

Now let's make a suitable problem.

In [ ]:
n = 3
J = - np.ones(n**2).reshape(n,n)
h = np.zeros(n)

It is a pair of three qubits that interact with each other.
However, let|01> and|10> be in a low energy (gain) state and let|00> and|11> be in a high energy (loss) state. This is called the antiferromagnetic Ising model.
This creates a state of frustration, where all spins are in a 3-spin state, and it is hard to know which one to point up.

In theoretical quantum annealing, it is better to make the change little by little, so vary the strength of the rotating X and Z related gates by k/Tall and (1-k/Tall) while applying very small dt.

In [ ]:
#Number of Steps
Tall = 100
dt = 0.01

qc10 = cirq.Circuit()
q = cirq.LineQubit.range(n)

#Start with a superposition
qc10.append(cirq.H.on_each(q))

for k in range(Tall):
  s1 = dt*(1 - k/Tall)
  s2 = dt*k/Tall
  Uqubo = QA(J, h, s1, s2)
  qc10.append(Uqubo.on(*q))

In [ ]:
state10 = sim_state(qc10)

In [ ]:
def binary_labels(num_qubits):
    return [bin(x)[2:].zfill(num_qubits) for x in range(2 ** num_qubits)]

ans10 = sim_state_exp(qc10)
cirq.plot_state_histogram(ans10, plt.subplot(), tick_label=binary_labels(n))
plt.show()

This shows that certain states have a high probability of being selected.
We can see which ones to choose, such as|001>,|010>, and|100>, and which ones to choose.
And also candidates such as|011>,|010>, and|110> are appearing.

### QAOA (Quantum Approximation Optimization Algorithm)

Now, QAOA is a method to obtain an efficient optimal solution by optimizing the time to act on the rotating X-gate and the Z-related gate of quantum annealing.
The criterion for optimization is energy.

The criterion for that optimization is energy, and it is necessary to be able to calculate that energy from the output result of the quantum circuit.

Calculate the energy in terms of the expected value of the spin we used last time.

In [ ]:
def ene_exp(qc,h,J,n):
  state = sim_state(qc, disp=False)

  op = cirq.PauliString()
  for k in range(n):
    op += -float(h[k])*cirq.Z(q[k])
 
  for k in range(n):
    for l in range(n):
      if k < l:
        op += -float(J[k,l])*cirq.Z(q[k])*cirq.Z(q[l])
      elif k > l:
        op += -float(J[l,k])*cirq.Z(q[l])*cirq.Z(q[k])

  collector = cirq.PauliSumCollector(circuit=qc, observable=op, samples_per_term=100)
  collector.collect(sampler=cirq.Simulator())
  y = collector.estimated_energy()

  return y, state

Optimize parameters along the way based on this.

In [ ]:
def ene_func(params):
  Tall = int(len(params)/2)
  qc = cirq.Circuit()
  q = cirq.LineQubit.range(n)

  #Start with a superposition
  qc.append(cirq.H.on_each(q))

  for k in range(Tall):
    s1 = params[k]
    s2 = params[k+Tall]
    Uqubo = QA(J, h, s1, s2)
    qc.append(Uqubo.on(*q))
    
  ene = ene_exp(qc,h,J,n)

  return ene

In [ ]:
Tall = 2
params = np.random.rand(2*Tall)

Let's use the same optimization method as before, but without gradients.

In [ ]:
from scipy.optimize import minimize
result = minimize(ene_func, params, method="COBYLA", options={"maxiter": 100})

To get the resulting parameters, it was result.x, wasn't it?

In [ ]:
result.fun

In [ ]:
result.x

What if we actually run it with the parameters obtained?

In [ ]:
params = result.x
qc11 = cirq.Circuit()
q = cirq.LineQubit.range(n)

#Start with a superposition
qc11.append(cirq.H.on_each(q))

for k in range(Tall):
  s1 = params[k]
  s2 = params[k+Tall]
  Uqubo = QA(J, h, s1, s2)
  qc11.append(Uqubo.on(*q))

In [ ]:
ans11 = sim_state_exp(qc11)
cirq.plot_state_histogram(ans11, plt.subplot(), tick_label=binary_labels(n))
plt.show()

You have obtained a quantum circuit that can successfully obtain the desired state.
In terms of the number of steps, you obtained something less than what was used in the quantum annealing simulation.

ちなみに同じ問題を株式会社Jijの開発する量子アニーリングシミュレータのOpenJijで解かせてみるとどうでしょうか。


In [ ]:
pip install openjij

These include simulations of quantum annealing (using the quantum Monte Carlo method).

In [ ]:
from openjij import SQASampler
sampler = SQASampler()

h and J are processed as follows for input in dict format.
(The sign of the interaction in the QA simulator is opposite to that in physics textbooks, etc., so the sign is minus.)

In [ ]:
h_dict = {}
for i in range(n):
  h_dict[i] = h[i]

J_dict = {}
for i in range(n):
  for j in range(n):
    if i != j:
      J_dict[i,j] = - J[i,j]

Execution is simple, just throw h and J as follows.

In [ ]:
sampleset = sampler.sample_ising(h_dict, J_dict, num_reads=10)

In [ ]:
print(sampleset.record)

Let's try to solve each of these problems with a more concrete problem.
We will deal with a number division problem.
You have some numbers and you want to divide them into two numbers.
However, we want the sum of the two numbers to be as equal as possible.
If the difference between the sums of the two numbers is small, then the answer is correct.
When each number is divided into two groups A and B as $n_i$, the sum of each number is calculated as follows.

\begin{equation}
I_A = \sum_{i \in A} n_i
\end{equation}
and
\begin{equation}
I_B = \sum_{i \in B} n_i
\end{equation}

The smaller these differences are, whether positive or negative, the happier we are, so we take the difference and square it.
\begin{equation}
(I_A - I_B)^2 = (\sum_{i \in A} n_i
- \sum_{i \in B} n_i)^2
\end{equation}
Consider dividing them into these two groups by using the binary values of 0 and 1 or -1 and +1, as in the Ising model. writing again those assigned to A as z_i=+1 and those assigned to B as z_i=-1, we can see that this The situation can be written in a mathematical formula.
\begin{equation}
(I_A - I_B)^2 = (\sum_{i} n_i z_i )^2
\end{equation}
We can think of a minimization problem for this.
Let's expand on that a bit.
\begin{equation}
(\sum_{i} n_i z_i )(\sum_{j} n_j z_j ) = 2\sum_i n_i + \sum_{i \neq j} n_in_j z_i z_j 
\end{equation}
and that it has the same form as the Ising model.



In [ ]:
n = 6
N = np.linspace(1,n,n)

First, prepare numbers from 1 to n.
Multiply these together to make J (no h is fine this time)

In [ ]:
h = np.zeros(n)
J = np.zeros(n**2).reshape(n,n)

for k in range(n):
  for l in range(n):
    if k != l:
      J[k,l] = - N[k]*N[l]

First, let's run it with quantum annealing.

In [ ]:
#Number of Steps
Tall = 200
dt = 0.01

qc12 = cirq.Circuit()
q = cirq.LineQubit.range(n)

#Start with a superposition
qc12.append(cirq.H.on_each(q))

for k in range(Tall):
  s1 = dt*k/Tall
  s2 = dt*(1 - k/Tall)
  Uqubo = QA(J, h, s1, s2)
  qc12.append(Uqubo.on(*q))

In [ ]:
state12 = sim_state(qc12)

In [ ]:
ans12 = sim_state_exp(qc12)
cirq.plot_state_histogram(ans12, plt.subplot(), tick_label=binary_labels(n))
plt.xticks(rotation=-90)
plt.show()

It is hard to see in the graph, but several states are highlighted.
The ans can be seen in the numerical values.

Let's do the same with QAOA.

In [ ]:
def ene_func(params):
  Tall = int(len(params)/2)
  qc = cirq.Circuit()
  q = cirq.LineQubit.range(n)

  #Start with a superposition
  qc.append(cirq.H.on_each(q))

  for k in range(Tall):
    s1 = params[k]
    s2 = params[k+Tall]
    Uqubo = QA(J, h, s1, s2)
    qc.append(Uqubo.on(*q))
    
  ene = ene_exp(qc,h,J,n)

  return ene

In [ ]:
Tall = 10
params = np.random.rand(2*Tall)

In [ ]:
from scipy.optimize import minimize
result = minimize(ene_func, params, method="COBYLA", options={"maxiter": 100})

If you want to extract the obtained results, use result.fun and result.x.

In [ ]:
result.fun

In [ ]:
params = result.x
qc13 = cirq.Circuit()
q = cirq.LineQubit.range(n)

#Start with a superposition
qc13.append(cirq.H.on_each(q))

for k in range(Tall):
  s1 = params[k]
  s2 = params[k+Tall]
  Uqubo = QA(J, h, s1, s2)
  qc13.append(Uqubo.on(*q))

In [ ]:
ans13 = sim_state_exp(qc13)
cirq.plot_state_histogram(ans13, plt.subplot(), tick_label=binary_labels(n))
plt.xticks(rotation=90)
plt.show()

It is quite a difficult problem, and if you can find a division 2,3,5 and 1,4,6 or something like that with a total of 10 and 11, you are correct.
1,4,5, 2,3,6, etc. are also acceptable.

In [ ]:
ans13.histogram(key='m')